In [1]:
import numpy as np
import requests
import os
import pandas as pd
from selenium import webdriver
import time
import copy
import urllib3
from bs4 import BeautifulSoup
import re
import csv
from selenium import webdriver

In [2]:
def preprocess_match(game, team_name, fileName):
    matchID_set = [str(x) for x in range(0, 200)]
    game.rename(columns = {'Unnamed: 5':'Home_away'}, inplace = True)
    mask_home = (game['Home_away'] == '@')
    mask_away = (game['Home_away'] != '@')
    game.loc[mask_home, 'Home_away'] = 1
    game.loc[mask_away, 'Home_away'] = 0
    game = game.loc[game['G'].isin(matchID_set)]
    game = game[['G', 'Opponent', 'Home_away','Tm', 'Opp']]
    file_path = './match_data/' + team_name 
    os.makedirs(file_path, exist_ok=True)
    game.to_csv(file_path + '/' + fileName + '.csv')
def collect_match(start_year, end_year, team_name, dict_used_name):
    year_list = [str(x) for x in range(start_year, end_year)]
    for year in year_list:
        for team in dict_used_name:
            url = 'https://www.basketball-reference.com/teams/' + team +'/' + year + '_games.html'
            try:
                tb = pd.read_html(url)
                fileName = team_name +'_'+ year + '_regular'
                preprocess_match(tb[0], team_name,  fileName)
                try :
                    tb[1]
                except:
                    continue
                fileName = team_name +'_'+ year + '_playoff'
                preprocess_match(tb[1], team_name, fileName)
            except:
                continue
             
def collect_coach_id():
    coach_id = []
    res = requests.get('https://www.basketball-reference.com/coaches/NBA_stats.html/')
    regular_exp = '.*><a href="/coaches/' + charc + '/(.*)">.*'
    result = re.findall(regular_exp, res.text)
    for x in result:
        coach_id.append(x.split(".html")[0])
    file = open('./coach_data/coach_id.txt', 'w')
    for cid in coach_id:
        file.write("%s\n" % cid)
def collect_player_id():
    alphabet = [chr(i) for i in range(97,123)] 
    player_id = []
    for charc in alphabet:
        res = requests.get('https://www.basketball-reference.com/players/' + charc +'/')
        regular_exp = '.*><a href="/players/' + charc + '/(.*)">.*'
        result = re.findall(regular_exp, res.text)
        for x in result:
            player_id.append(x.split(".html")[0])
    file = open('./player_data/player_id.txt', 'w')
    for pid in player_id:
        file.write("%s\n" % pid)
def read_player_id():
    with open('./player_data/player_id.txt') as f:
        content = f.readlines()
    content = [x.strip() for x in content] 
    return content
def collect_player_basic():
    alphabet = [chr(i) for i in range(97,123)] 
    for charc in alphabet:
        if charc != 'x':
            url = 'https://www.basketball-reference.com/players/' + charc + '/'
            tb = pd.read_html(url)
            for k in range(len(tb[0]['Ht'])):
                tb[0]['Ht'][k] = str(tb[0]['Ht'][k]) + '\t'
            file_path = './player_data/basic/' 
            os.makedirs(file_path, exist_ok=True)
            tb[0].to_csv(file_path + '/' + charc + '.csv')     
def collect_player_performance():
#     player_id = read_player_id()
    file_path = './player_data/performance/regular/'
    player_id = ['abdelal01', 'abdulza01', 'abdulka01', 'abdulma02', 'abdulta01', 'abdursh01', 'abernto01', 'ablefo01', 'abramjo01', 'abrinal01',
            'bakerma01', 'jamesle01', 'bakerno01', 'bakerro01', 'bakervi01', 'baldwwa01', 'balkmre01', 'ballce01', 'balllo01', 'ballagr01', 'baltihe01', 'banksge01',
             'jacksbo01', 'jacksce01', 'jacksda01', 'jacksde01', 'jacksgr01', 'jacksja01', 'jacksje01', 'jacksji01', 'jacksjo02', 'jacksju01']
    all_tab = []
    count = 0
    for year in range(2015, 2018):
        for pid in player_id:
            
#             print('https://www.basketball-reference.com/players/' + pid[0] +'/' + str(pid) + '/gamelog/' + str(year) +'/')
            try:
                url = 'https://www.basketball-reference.com/players/' + pid[0] +'/' + str(pid) + '/gamelog/' + str(year) +'/'
                tb = pd.read_html(url)
                tb = tb[7]
                tb.insert(loc=0, column='Id', value= [pid] * len(tb))
                if count == 0:
                    all_tab = copy.deepcopy(tb)
                    count += 1
                else:
                    all_tab = all_tab.append(copy.deepcopy(tb))
                print(year, pid)
            except:
                continue
        if not all_tab.empty:
            os.makedirs(file_path, exist_ok=True)
            all_tab = all_tab.fillna(0)
            all_tab.to_csv(file_path + str(year) + '.csv')
            count = 0
        else:
            print(year, ' is empty')
        
# def collect_player_average():
#     player_id = read_player_id()
#     for pid in player_id:

In [11]:
player_id = ['jamesle01']
all_tab = []
count = 0
for year in range(1999, 2018):
    for pid in player_id:

#             print('https://www.basketball-reference.com/players/' + pid[0] +'/' + str(pid) + '/gamelog/' + str(year) +'/')
#         try:
            url = 'https://www.basketball-reference.com/players/' + pid[0] +'/' + str(pid) + '/gamelog/' + str(year) +'/'
            tb = pd.read_html(url)
            tb = tb[7]
            tb.insert(loc=0, column='Id', value= [pid] * len(tb))
            if count == 0:
                all_tab = copy.deepcopy(tb)
                count += 1
            else:
                all_tab = all_tab.append(copy.deepcopy(tb))
            print(year, pid)
    if all_tab.empty:
            os.makedirs(file_path, exist_ok=True)
            all_tab = all_tab.fillna(0)
            all_tab.to_csv(file_path + str(year) + '.csv')
            count = 0
    else:
            print(year, ' is empty')

2015 jamesle01
2015  is empty
2016 jamesle01
2016  is empty
2017 jamesle01
2017  is empty


In [15]:
collect_player_performance()

2015 jamesle01
2016 jamesle01
2017 abrinal01
2017 jamesle01
2017 bakerro01
2017 baldwwa01
2017 jacksde01


In [11]:
# collect_player_performance()
# https://www.basketball-reference.com/players/c/curryst01.html
player_id = read_player_id()
count = 0
for pid in player_id:
    try:
        url = 'https://www.basketball-reference.com/players/'+pid[0]+'/'+pid+'.html'
        tb = pd.read_html(url)
        col_ct = 0
        lst = (list(tb[0])[0:30])
        for it in lst:
            if it.split(':')[0] != 'Unnamed':
                col_ct += 1
            else:
                break
        tb = tb[0][list(tb[0])[0:col_ct]]
        tb.insert(loc=0, column='Id', value= [pid] * len(tb['Season']))
        tb = tb[0:-1]
        if count == 0:
            all_tab = copy.deepcopy(tb)
            count += 1
        else:
            all_tab = all_tab.append(copy.deepcopy(tb))
        print(pid)
    except:
        continue
all_tab = all_tab.fillna(0)
all_tab.to_csv('player_perform_avg.csv')

abdelal01


In [10]:
all_tab = all_tab.fillna(0)
all_tab['3P%']

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
Name: 3P%, dtype: float64

In [3]:
dict_used_name = {}
dict_used_name['ATL'] = ['ATL','STL', 'MLH']
dict_used_name['BOS'] = ['BOS']
dict_used_name['BRK'] = ['BRK', 'NJN', 'NYN','NJA','NYA']
dict_used_name['CHO'] = ['CHO','CHH','CHA']
dict_used_name['CHI'] = ['CHI']
dict_used_name['CLE'] = ['CLE']
dict_used_name['DAL'] = ['DAL']
dict_used_name['DEN'] = ['DEN', 'DNR','DNA']
dict_used_name['DET'] = ['DET', 'FTW']
dict_used_name['GSW'] = ['GSW', 'SFW', 'PHW']
dict_used_name['HOU'] = ['HOU', 'SDR']
dict_used_name['IND'] = ['IND', 'INA']
dict_used_name['LAC'] = ['LAC', 'SDC', 'BUF']
dict_used_name['LAL'] = ['LAL', 'MNL']
dict_used_name['MEM'] = ['MEM', 'VAN']
dict_used_name['MIA'] = ['MIA']
dict_used_name['MIL'] = ['MIL']
dict_used_name['MIN'] = ['MIN']
dict_used_name['NOP'] = ['NOP', 'NOK','NOH']
dict_used_name['NYK'] = ['NYK']
dict_used_name['OKC'] = ['SEA', 'OKC']
dict_used_name['ORL'] = ['ORL']
dict_used_name['PHI'] = ['PHI', 'SYR']
dict_used_name['PHO'] = ['PHO']
dict_used_name['POR'] = ['POR']
dict_used_name['SAS'] = ['SAS', 'DLC', 'TEX', 'SAA']
dict_used_name['SAC'] = ['SAC', 'KCK','KCO','CIN','ROC']
dict_used_name['TOR'] = ['TOR']
dict_used_name['UTA'] = ['UTA', 'NOJ']
dict_used_name['WAS'] = ['WAS', 'WSB','CAP','BAL','CHZ','CHP']
team_name_list = ['ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 
                  'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 
                  'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAS', 'SAC', 'TOR', 'UTA', 'WAS']



In [28]:
count = 0
for k in team_name_list:
#     all_tb = pd.DataFrame()
    for year in range(1950, 2018):
        for term in ['playoff', 'regular']:
            try:
                FileName = (k + '_' + str(year) + '_' + term)
                dfm = pd.read_csv('./match_data/' + k + '/' + FileName + '.csv', encoding="ISO-8859-1")
#                 print(dfm.columns.get_values().tolist())
                dfm.insert(loc=0, column='year', value= [year] * len(dfm))
                dfm.insert(loc=0, column='team', value= [k] * len(dfm))
                dfm.insert(loc=0, column='playoff_regualr', value= [term] * len(dfm))
                if count == 0:
#                     print(dfm)
                    all_tb = copy.deepcopy(dfm)
                    count += 1
                else:
                    all_tb = all_tb.append(copy.deepcopy(dfm))
                    print(year, k)
            except:
#                 print(year, k)
                continue
#     all_tb.drop(all_tb.columns[4], axis=1)
#     print(all_tb)
all_tb.to_csv('./match_data/' + 'allgame' +'.csv')
count = 0
            

1953 ATL
1954 ATL
1955 ATL
1956 ATL
1956 ATL
1957 ATL
1957 ATL
1958 ATL
1958 ATL
1959 ATL
1959 ATL
1960 ATL
1960 ATL
1961 ATL
1961 ATL
1962 ATL
1963 ATL
1963 ATL
1964 ATL
1964 ATL
1965 ATL
1965 ATL
1966 ATL
1966 ATL
1967 ATL
1967 ATL
1968 ATL
1968 ATL
1969 ATL
1969 ATL
1970 ATL
1970 ATL
1971 ATL
1971 ATL
1972 ATL
1972 ATL
1973 ATL
1973 ATL
1974 ATL
1975 ATL
1976 ATL
1977 ATL
1978 ATL
1978 ATL
1979 ATL
1979 ATL
1980 ATL
1980 ATL
1981 ATL
1982 ATL
1982 ATL
1983 ATL
1983 ATL
1984 ATL
1984 ATL
1985 ATL
1986 ATL
1986 ATL
1987 ATL
1987 ATL
1988 ATL
1988 ATL
1989 ATL
1989 ATL
1990 ATL
1991 ATL
1991 ATL
1992 ATL
1993 ATL
1993 ATL
1994 ATL
1994 ATL
1995 ATL
1995 ATL
1996 ATL
1996 ATL
1997 ATL
1997 ATL
1998 ATL
1998 ATL
1999 ATL
1999 ATL
2000 ATL
2001 ATL
2002 ATL
2003 ATL
2004 ATL
2005 ATL
2006 ATL
2007 ATL
2008 ATL
2008 ATL
2009 ATL
2009 ATL
2010 ATL
2010 ATL
2011 ATL
2011 ATL
2012 ATL
2012 ATL
2013 ATL
2013 ATL
2014 ATL
2014 ATL
2015 ATL
2015 ATL
2016 ATL
2016 ATL
2017 ATL
2017 ATL
1950 BOS
1

2012 HOU
2013 HOU
2013 HOU
2014 HOU
2014 HOU
2015 HOU
2015 HOU
2016 HOU
2016 HOU
2017 HOU
2017 HOU
1968 IND
1968 IND
1969 IND
1969 IND
1970 IND
1970 IND
1971 IND
1971 IND
1972 IND
1972 IND
1973 IND
1973 IND
1974 IND
1974 IND
1975 IND
1975 IND
1976 IND
1976 IND
1977 IND
1978 IND
1979 IND
1980 IND
1981 IND
1981 IND
1982 IND
1983 IND
1984 IND
1985 IND
1986 IND
1987 IND
1987 IND
1988 IND
1989 IND
1990 IND
1990 IND
1991 IND
1991 IND
1992 IND
1992 IND
1993 IND
1993 IND
1994 IND
1994 IND
1995 IND
1995 IND
1996 IND
1996 IND
1997 IND
1998 IND
1998 IND
1999 IND
1999 IND
2000 IND
2000 IND
2001 IND
2001 IND
2002 IND
2002 IND
2003 IND
2003 IND
2004 IND
2004 IND
2005 IND
2005 IND
2006 IND
2006 IND
2007 IND
2008 IND
2009 IND
2010 IND
2011 IND
2011 IND
2012 IND
2012 IND
2013 IND
2013 IND
2014 IND
2014 IND
2015 IND
2016 IND
2016 IND
2017 IND
2017 IND
1971 LAC
1972 LAC
1973 LAC
1974 LAC
1974 LAC
1975 LAC
1975 LAC
1976 LAC
1976 LAC
1977 LAC
1978 LAC
1979 LAC
1980 LAC
1981 LAC
1982 LAC
1983 LAC
1984 LAC
1

1975 POR
1976 POR
1977 POR
1977 POR
1978 POR
1978 POR
1979 POR
1979 POR
1980 POR
1980 POR
1981 POR
1981 POR
1982 POR
1983 POR
1983 POR
1984 POR
1984 POR
1985 POR
1985 POR
1986 POR
1986 POR
1987 POR
1987 POR
1988 POR
1988 POR
1989 POR
1989 POR
1990 POR
1990 POR
1991 POR
1991 POR
1992 POR
1992 POR
1993 POR
1993 POR
1994 POR
1994 POR
1995 POR
1995 POR
1996 POR
1996 POR
1997 POR
1997 POR
1998 POR
1998 POR
1999 POR
1999 POR
2000 POR
2000 POR
2001 POR
2001 POR
2002 POR
2002 POR
2003 POR
2003 POR
2004 POR
2005 POR
2006 POR
2007 POR
2008 POR
2009 POR
2009 POR
2010 POR
2010 POR
2011 POR
2011 POR
2012 POR
2013 POR
2014 POR
2014 POR
2015 POR
2015 POR
2016 POR
2016 POR
2017 POR
2017 POR
1968 SAS
1968 SAS
1969 SAS
1969 SAS
1970 SAS
1970 SAS
1971 SAS
1971 SAS
1972 SAS
1972 SAS
1973 SAS
1974 SAS
1974 SAS
1975 SAS
1975 SAS
1976 SAS
1976 SAS
1977 SAS
1977 SAS
1978 SAS
1978 SAS
1979 SAS
1979 SAS
1980 SAS
1980 SAS
1981 SAS
1981 SAS
1982 SAS
1982 SAS
1983 SAS
1983 SAS
1985 SAS
1985 SAS
1986 SAS
1986 SAS
1

In [ ]:
for f

In [27]:
pid_list = read_player_id()

In [9]:
collect_player_basic()

/Users/zhoutongjiang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [81]:

driver = webdriver.Chrome()
driver.get('https://www.basketball-reference.com/players/z/zellety01/gamelog/2017/')
soup = BeautifulSoup(driver.page_source,'lxml')
driver.quit()
table = soup.find_all('table')[-1]  #This is the index of any table of that page. If you change it you can get different tables.
# print(table[])
tab_data = [[celldata.text for celldata in rowdata.find_all(["th","td"])]
                        for rowdata in table.find_all("tr")]

with open("./player_data/performance/test.csv",'w') as resultFile:
    wr = csv.writer(resultFile, dialect='excel')
    for k in tab_data:
        wr.writerow(k)

Rk G Date Age Tm   Opp   GS MP FG FGA FG% 3P 3PA 3P% FT FTA FT% ORB DRB TRB AST STL BLK TOV PF PTS GmSc +/-
1 1 2017-04-16 27-089 BOS  CHI L (-4) 0 3:03 0 1 .000 0 0  0 0  1 1 2 0 0 0 0 1 0 -0.1 +1
2 2 2017-04-18 27-091 BOS  CHI L (-14) 0 8:39 2 2 1.000 0 0  0 0  1 0 1 0 0 0 1 0 4 3.1 +1
3  2017-04-21 27-094 BOS @ CHI W (+17) Did Not Play
4  2017-04-23 27-096 BOS @ CHI W (+9) Did Not Play
5  2017-04-26 27-099 BOS  CHI W (+11) Did Not Play
6 3 2017-04-28 27-101 BOS @ CHI W (+22) 0 5:09 0 1 .000 0 0  0 0  0 2 2 0 0 0 0 0 0 -0.1 -5
7  2017-04-30 27-103 BOS  WAS W (+12) Did Not Play
8  2017-05-02 27-105 BOS  WAS W (+10) Did Not Play
9 4 2017-05-04 27-107 BOS @ WAS L (-27) 0 10:34 0 3 .000 0 0  3 4 .750 0 4 4 1 0 0 0 1 3 2.0 -6
10 5 2017-05-07 27-110 BOS @ WAS L (-19) 0 5:27 2 3 .667 0 0  0 0  0 0 0 1 0 0 0 1 4 3.0 +3
11 6 2017-05-10 27-113 BOS  WAS W (+22) 0 4:21 2 5 .400 0 0  2 2 1.000 2 0 2 0 0 0 0 1 6 4.3 -2
12  2017-05-12 27-115 BOS @ WAS L (-1) Did Not Play
13  2017-05-15 27-118 BOS  

In [33]:
# options.add_argument("--test-type")
# options.binary_location = "/usr/bin/chromium"
# # driver = webdriver.Chrome()
# driver.get('https://www.basketball-reference.com/players/a/')

In [34]:
# driver = webdriver.Chrome()


In [49]:
alphabet = [chr(i) for i in range(97,123)] 
# for ch in alphabet:
#     try:
#         df = pd.read_csv('./player_data/performance/avg_perform/'+ch+'.csv', encoding="ISO-8859-1")
#         df.drop(df.columns[[0]],axis=1,inplace=True)
#         df.to_csv('./player_data/performance/avg_perform/'+ch+'1.csv' ,  index = False)
#         print(df)
#     except:
#         pass
df = pd.read_csv('./player_data/performance/avg_perform/'+'a'+'.csv', encoding="ISO-8859-1")
for ch in alphabet[1:]:
    try:
        dfm = copy.deepcopy(pd.read_csv('./player_data/performance/avg_perform/'+ch+'.csv', encoding="ISO-8859-1"))
        df = df.append(copy.deepcopy(dfm))
        print(ch)
    except:
        pass
df.to_csv('./player_data/performance/avg_perform/all.csv' ,  index = False)

b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
y
z


In [57]:
for ch in range(1999,2000):
#     try:
        dfm = copy.deepcopy(pd.read_csv('./player_data/performance/regular/'+str(ch)+'.csv', encoding="ISO-8859-1"))
#         df = df.append(copy.deepcopy(dfm))
#         dfm.drop(df.columns[[1]],axis=1,inplace=True)
        dfm.to_csv('./player_data/performance/regular/'+str(ch)+'1.csv' ,  index = False)
        print('!')
#     except:
#         pass

!


In [2]:
# file_path = './team_data/' + 'AHL'
# print(file_path)
# os.makedirs(file_path)

In [3]:
url = 'https://www.basketball-reference.com/players/z/zellety01/gamelog/2017/#pgl_basic_playoffs::none'
tb = pd.read_html(url)

,Rk,G,Date,Age,Tm,,Opp,.1,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,1,2016-10-26,26-283,BOS,NaN,BRK,W (+5),0,16:26,...,2,4,3,0,2,1,2,13,13.8,-8
1,2,2,2016-10-27,26-284,BOS,@,CHI,L (-6),0,20:18,...,4,5,1,0,2,1,3,6,2.3,+2
2,3,3,2016-10-29,26-286,BOS,@,CHO,W (+6),0,12:04,...,4,4,1,0,1,0,3,2,3.1,-4
3,4,4,2016-11-02,26-290,BOS,NaN,CHI,W (+7),1,24:45,...,1,4,0,1,0,0,3,11,7.1,+9
4,5,5,2016-11-03,26-291,BOS,@,CLE,L (-6),1,27:46,...,7,10,6,0,2,0,3,11,15.6,-7
5,6,6,2016-11-06,26-294,BOS,NaN,DEN,L (-16),1,25:46,...,5,5,2,1,1,1,5,6,3.9,-12
6,7,7,2016-11-09,26-297,BOS,@,WAS,L (-25),1,14:18,...,1,2,1,0,1,0,0,6,6.8,-6
7,8,8,2016-11-11,26-299,BOS,NaN,NYK,W (+28),0,20:49,...,3,4,0,2,1,3,4,11,6.9,+15
8,9,9,2016-11-12,26-300,BOS,@,IND,W (+6),0,15:46,...,2,6,0,0,1,0,0,7,7.7,+7
9,10,10,2016-11-14,26-302,BOS,@,NOP,L (-1),0,5:07,...,0,0,1,0,0,0,0,0,-0.7,-7


In [3]:
# import urllib.request
with urllib.request.urlopen("https://www.basketball-reference.com/players/z/zellety01/gamelog/2017/") as url:
    html_text = url.read()
# print(html_text)
from bs4 import BeautifulSoup, Comment
soup = BeautifulSoup(html_text, 'lxml')
comments = soup.findAll(text=lambda text:isinstance(text, Comment))
[comment.extract() for comment in comments]

NameError: name 'urllib' is not defined

In [60]:
type(html_text)

bytes

In [4]:
alphabet = [chr(i) for i in range(97,98)] 
for charc in alphabet:
    if charc != 'x':
        url = 'https://www.basketball-reference.com/players/' + charc + '/'
        tb = pd.read_html(url)
        for k in range(len(tb[0]['Ht'])):
            tb[0]['Ht'][k] = str(tb[0]['Ht'][k]) + '\t'
        file_path = './player_data/basic/' 
        os.makedirs(file_path, exist_ok=True)
        tb[0].to_csv(file_path + '/' + charc + '.csv')     



/Users/zhoutongjiang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [5]:
url = 'https://www.basketball-reference.com/players/' + 'a' + '/'
tb = pd.read_html(url)

In [6]:
tb

[                   Player  From    To  Pos    Ht   Wt          Birth Date  \
 0          Alaa Abdelnaby  1991  1995  F-C  6-10  240       June 24, 1968   
 1         Zaid Abdul-Aziz  1969  1978  C-F   6-9  235       April 7, 1946   
 2    Kareem Abdul-Jabbar*  1970  1989    C   7-2  225      April 16, 1947   
 3      Mahmoud Abdul-Rauf  1991  2001    G   6-1  162       March 9, 1969   
 4       Tariq Abdul-Wahad  1998  2003    F   6-6  223    November 3, 1974   
 5     Shareef Abdur-Rahim  1997  2008    F   6-9  225   December 11, 1976   
 6           Tom Abernethy  1977  1981    F   6-7  220         May 6, 1954   
 7             Forest Able  1957  1957    G   6-3  180       July 27, 1932   
 8          John Abramovic  1947  1948    F   6-3  195    February 9, 1919   
 9            Alex Abrines  2017  2018  G-F   6-6  190      August 1, 1993   
 10             Alex Acker  2006  2009    G   6-5  185    January 21, 1983   
 11           Don Ackerman  1954  1954    G   6-0  183   Septemb